In [1]:
%load_ext tensorboard

import random
from pathlib import Path
import glob
import os
from tqdm import tqdm
import numpy as np
import pdb
import time


import torch
from torch import nn
import torchvision as TV
import torch.nn.functional as F
import torchvision.transforms as T
from torch.nn import UpsamplingNearest2d
from torch.nn.utils import spectral_norm
from torch.distributions import Normal
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights


import utils
import vae
import u_net

path_data=r"C:\Users\pauli\Desktop\UNI\UTRECHT\3RD PERIOD COURSES\CAPITA_SELECTA\group project\TrainingData\TrainingData"


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) #Device used: using GPU will significantly speed up the training.


cpu


using a module for transfer learning

In [2]:
# pip install segmentation-models-pytorch

import segmentation_models_pytorch as smp
unet_model = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

ModuleNotFoundError: No module named 'segmentation_models_pytorch'

### USE THIS: use the unet code from class


In [ ]:
# transfer learning
resnet = TV.models.resnet50(weights=ResNet50_Weights.DEFAULT)
sd = resnet.state_dict()

#initialise model with weights of resnet
unet_model = u_net.UNet(num_classes=1)
unet_model.state_dict(sd)

saw that the layer names were not the same, try another thing. This does not work

In [ ]:
unet_encoder_layers = list(unet_model.children())
resnet_encoder_layers = list(resnet.children())

for i, layer in enumerate(resnet_encoder_layers):
    print(layer.state_dict())
    unet_encoder_layers[i].load_state_dict(layer.state_dict())

### load paths and datasets

In [3]:
# to ensure reproducible training/validation split
random.seed(42)

# directorys with data and to store training checkpoints and logs
DATA_DIR = Path.cwd() / "TrainingData"
CHECKPOINTS_DIR = Path.cwd() / "segmentation_model_weights"
CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)
TENSORBOARD_LOGDIR = "segmentation_runs"

# training settings and hyperparameters
NO_VALIDATION_PATIENTS = 2
IMAGE_SIZE = [64, 64]
BATCH_SIZE = 32
N_EPOCHS = 40
LEARNING_RATE = 1e-4
TOLERANCE = 0.01  # for early stopping


In [4]:
# load data
# find patient folders in training directory
patients = [
    path
    for path in glob.glob(path_data+r"\p*[0-9]")
]

random.shuffle(patients)

# split in training/validation after shuffling
partition = {
    "train": patients[:-NO_VALIDATION_PATIENTS],
    "validation": patients[-NO_VALIDATION_PATIENTS:],
}


In [5]:
# load training data and create DataLoader with batching and shuffling
dataset = utils.ProstateMRDataset(partition["train"], IMAGE_SIZE)
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    pin_memory=True,
)

# load validation data
valid_dataset = utils.ProstateMRDataset(partition["validation"], IMAGE_SIZE)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    pin_memory=True,
)

### initialise optimiser and loss function

In [6]:
loss_function = utils.DiceBCELoss()
optimizer = torch.optim.Adam(unet_model.parameters(), lr=LEARNING_RATE)

minimum_valid_loss = 10  # initial validation loss
writer = SummaryWriter(log_dir=TENSORBOARD_LOGDIR)  # tensorboard summary

In [7]:
# training loop
startTime = time.time()

for epoch in range(N_EPOCHS):
    current_train_loss = 0.0
    current_valid_loss = 0.0

    # training iterations
    # tqdm is for timing iteratiions
    for inputs, labels in tqdm(dataloader, position=0):
        # needed to zero gradients in each iterations
        optimizer.zero_grad()
        outputs = unet_model(inputs)  # forward pass
        loss = loss_function(outputs, labels.float())
        loss.backward()  # backpropagate loss
        current_train_loss += loss.item()
        optimizer.step()  # update weights

    # evaluate validation loss
    with torch.no_grad():
        unet_model.eval()  # turn off training option for evaluation
        for inputs, labels in tqdm(valid_dataloader, position=0):
            outputs = unet_model(inputs)  # forward pass
            loss = loss_function(outputs, labels.float())
            current_valid_loss += loss.item()

        unet_model.train()  # turn training back on
        
    
    print("[INFO] EPOCH: {}".format(epoch + 1))
    print("Train loss: {:.6f}, Val loss: {:.4f}".format(current_train_loss/ len(dataloader), current_valid_loss/ len(valid_dataloader)))

    # write to tensorboard log
    writer.add_scalar("Loss/train", current_train_loss / len(dataloader), epoch)
    writer.add_scalar(
        "Loss/validation", current_valid_loss / len(valid_dataloader), epoch
    )

    # if validation loss is improving, save model checkpoint
    # only start saving after 10 epochs
    if (current_valid_loss / len(valid_dataloader)) < minimum_valid_loss + TOLERANCE:
        minimum_valid_loss = current_valid_loss / len(valid_dataloader)
        if epoch > 9:
            torch.save(
                unet_model.cpu().state_dict(),
                CHECKPOINTS_DIR / f"u_net_{epoch}.pth",
            )
            
# display the total time needed to perform the training
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

100%|██████████| 5/5 [00:05<00:00,  1.05s/it]


[INFO] EPOCH: 1
Train loss: 1.375951, Val loss: 1.3782


100%|██████████| 5/5 [00:05<00:00,  1.12s/it]


[INFO] EPOCH: 2
Train loss: 1.098003, Val loss: 1.1664


100%|██████████| 5/5 [00:04<00:00,  1.10it/s]


...

[INFO] EPOCH: 38
Train loss: 0.063503, Val loss: 0.3800


100%|██████████| 5/5 [00:07<00:00,  1.43s/it]


[INFO] EPOCH: 39
Train loss: 0.060668, Val loss: 0.3812


100%|██████████| 5/5 [00:06<00:00,  1.23s/it]


[INFO] EPOCH: 40
Train loss: 0.059871, Val loss: 0.3720
[INFO] total time taken to train the model: 5344.76s


In [65]:
path=r"C:\Users\pauli\Desktop\UNI\UTRECHT\3RD PERIOD COURSES\CAPITA_SELECTA\group project\segmentation_runs"
from os import listdir
from os.path import isfile, join

images_names= [f for f in listdir(path) if isfile(join(path, f))]

print(images_names)

['events.out.tfevents.1679764552.DESKTOP-RB7PK2S.6808.0']


In [3]:
# plot the training and validation loss
#%tensorboard --logdir 'C:\Users\pauli\Desktop\UNI\UTRECHT\3RD PERIOD COURSES\CAPITA_SELECTA\group project\segmentation_runs'
%reload_ext tensorboard

%tensorboard --logdir '$TENSORBOARD_LOGDIR'


Reusing TensorBoard on port 6006 (pid 22408), started 2 days, 22:13:32 ago. (Use '!kill 22408' to kill it.)